#### Biblioteca Usadas:
* [Numpy](https://numpy.org/doc/) - Para calculos matriciais usando Python
* [ImgUtils](https://github.com/dimatura/imgutils) - Compilado de Libs para manipulação de imagem em Python
* [Open CV](https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html) - Pré Processamento das Imagens e Edge Detection
* [PyTorch](https://pytorch.org/get-started/locally/) - Para facilitar o uso Easy OCR
* [Easy OCR](https://github.com/JaidedAI/EasyOCR) - Extração de Texto de Imagens

In [ ]:
import easyocr

In [5]:
conda install scipy

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3\envs\DeepLearning

  added / updated specs:
    - scipy


The following NEW packages will be INSTALLED:

  m2w64-gcc-libgfor~ conda-forge/win-64::m2w64-gcc-libgfortran-5.3.0-6
  m2w64-gcc-libs     conda-forge/win-64::m2w64-gcc-libs-5.3.0-7
  m2w64-gcc-libs-co~ conda-forge/win-64::m2w64-gcc-libs-core-5.3.0-7
  m2w64-gmp          conda-forge/win-64::m2w64-gmp-6.1.0-2
  m2w64-libwinpthre~ conda-forge/win-64::m2w64-libwinpthread-git-5.0.0.4634.697f757-2
  msys2-conda-epoch  conda-forge/win-64::msys2-conda-epoch-20160418-1
  scipy              conda-forge/win-64::scipy-1.9.1-py39h316f440_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils

##### 1. Lendo a imagem, e convertendo para escala de cinza:

##### 2. Aplicando Filtros e marcando as bordas das imagens

##### 3. Encontrando contotrnos e aplicando mascára de camada

##### 4. Usando Easy OCR para ler o texto:

##### 5. Renderizando o Resultado: